In [61]:
import pandas as pd
import numpy as np

In [62]:
df = pd.read_csv(r'C:/Users/vishw/OneDrive/Desktop/Indian Startup Funding/startup_funding.csv', encoding='latin1')
df.head()

,Sr No,Date,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09-01-2020,BYJUS,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13-01-2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09-01-2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02-01-2020,wealth bucket,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02-01-2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


In [63]:
df.isnull().sum()

Sr No                   0
Date                    0
Startup Name            0
Industry Vertical     171
SubVertical           936
City  Location        180
Investors Name         24
InvestmentnType         4
Amount in USD         960
Remarks              2625
dtype: int64

In [64]:
df.columns = df.columns.str.strip()              # Remove leading/trailing spaces
df.columns = df.columns.str.lower()              # Lowercase everything
df.columns = df.columns.str.replace(' ', '_')    # Replace spaces with underscores
df.columns = df.columns.str.replace('[^a-z0-9_]', '', regex=True)  # Remove special characters


print(df.columns)

Index(['sr_no', 'date', 'startup_name', 'industry_vertical', 'subvertical',
       'city__location', 'investors_name', 'investmentntype', 'amount_in_usd',
       'remarks'],
      dtype='object')


In [74]:
#Handling Sub-Vertical using Industry vertical by fillna method
def fill_subvertical(row):
    if pd.isnull(row['subvertical']):
        sub = df[df['industry_vertical'] == row['industry_vertical']]['subvertical'].mode()
        if not sub.empty:
            return sub[0]
    return row['subvertical']

df['subvertical'] = df.apply(fill_subvertical, axis=1)

#print("Missing in Sub Vertical:", df['SubVertical'].isnull().sum())

df['subvertical'] = df['subvertical'].fillna('Unknown')

print("Missing in Sub Vertical:", df['subvertical'].isnull().sum())

Missing in Sub Vertical: 0


In [81]:
#Filling Industry vertical using 'fillna' method and 'SubVertical' Column
df['industry_vertical'] = df['subvertical'].fillna('Unknown')

In [75]:
#Handlling missing city as 'unknown'
df['city__location'] = df['city__location'].fillna('Unknown')


In [76]:
#Handlling missing Investor Name as 'unknown'
df['investors_name'] = df['investors_name'].fillna('Unknown')

In [69]:
#dropping rows with missing InvestmentType
df = df[df['investmentntype'].notnull()]
print("missing value in investment type:", df['investmentntype'].isnull().sum())

missing value in investment type: 0


In [82]:
#Handling 'Amount' cloumn using 'Median' method
if df['amount_in_usd'].dtype == 'object':
    df['amount_in_usd'].describe()
    df['amount_in_usd'] = df['amount_in_usd'].str.replace(',','')
    df['amount_in_usd'] = pd.to_numeric(df['amount_in_usd'], errors='coerce')

overall_median = df['amount_in_usd'].median()
def fill_amount(row):
    if pd.isnull(row['amount_in_usd']):
        median_amnt = df[df['investmentntype'] == row['investmentntype']]['amount_in_usd'].median()
        if pd.isnull(median_amnt):
            return overall_median
        return median_amnt
    return row['amount_in_usd']

df['amount_in_usd'] = df.apply(fill_amount, axis=1)


In [ ]:
df.drop('remarks',axis=1, inplace=True)
df.isnull().sum()

sr_no                0
date                 0
startup_name         0
industry_vertical    0
subvertical          0
city__location       0
investors_name       0
investmentntype      0
amount_in_usd        0
dtype: int64

In [83]:
df.to_csv('Updated_data.csv', index=False)
print("data saved to 'Updated_data'")

data saved to 'Updated_data'
